# Session 1 – ചാറ്റ് ബൂട്ട്‌സ്റ്റ്രാപ്പ് (Foundry Local)

ഈ നോട്ട്‌ബുക്ക് Foundry Local ബൂട്ട്‌സ്റ്റ്രാപ്പ് ചെയ്യുന്നു, ഇഷ്ടപ്പെട്ട മോഡൽ അലിയാസ് ഡൗൺലോഡ് ചെയ്യുന്നു, കൂടാതെ ഒരു സ്റ്റാൻഡേർഡ് ചാറ്റ് പൂർത്തീകരണവും ഒരു സ്ട്രീമിംഗ് ചാറ്റ് പൂർത്തീകരണവും നടത്തുന്നു.


# സീനാരിയോ
ഈ സെഷൻ ഫൗണ്ടറി ലോക്കലിലൂടെ ഒരു ലൊക്കൽ ചെറിയ ഭാഷാ മോഡൽ പ്രതികരിക്കാൻ ആവശ്യമായ ഏറ്റവും അടിസ്ഥാന കാര്യങ്ങൾ പരിചയപ്പെടുത്തുന്നു. നിങ്ങൾ ചെയ്യേണ്ടത്:
- SDK / ക്ലയന്റ് ഡിപ്പൻഡൻസികൾ ഇൻസ്റ്റാൾ ചെയ്യുക.
- ഒരു തിരഞ്ഞെടുക്കപ്പെട്ട അലിയാസിനായി (ഡിഫോൾട്ട്: `phi-4-mini`) ഫൗണ്ടറി ലോക്കൽ മാനേജർ ഇൻഷിയലൈസ് ചെയ്യുക.
- മോഡൽ മെറ്റാഡാറ്റയിൽ ഓപ്ഷണൽ ഫീൽഡുകൾ സഹിക്കാനുള്ള ഒരു പ്രതിരോധ മങ്കി-പാച്ച് പ്രയോഗിക്കുക.
- ഒരു സ്റ്റാൻഡേർഡ് ചാറ്റ് പൂർത്തീകരണ അഭ്യർത്ഥന അയയ്ക്കുക.
- ഒരു ടോക്കൺ-പ്രതി-ടോക്കൺ പ്രതികരണം സ്ട്രീം ചെയ്യുക.

ലക്ഷ്യം നിങ്ങളുടെ ലൊക്കൽ റൺടൈം & നെറ്റ്‌വർക്ക് പാത പരിശോധിച്ച് RAG, റൂട്ടിംഗ്, അല്ലെങ്കിൽ ഏജന്റുകളിലേക്ക് നീങ്ങുന്നതിന് മുമ്പ് സ്ഥിരീകരിക്കുക എന്നതാണ്.


### Explanation: Dependency Installation
ഈ ലഘു ചാറ്റ് ഫ്ലോയ്ക്ക് ആവശ്യമായ Python പാക്കേജുകൾ ഇൻസ്റ്റാൾ ചെയ്യുന്നു:
- `foundry-local-sdk`: ലോക്കൽ മോഡലുകളും സർവീസ് ലൈഫ്‌സൈക്കിളും മാനേജ് ചെയ്യുന്നു.
- `openai`: ചാറ്റ് പൂർത്തീകരണങ്ങൾക്ക് പരിചിതമായ ക്ലയന്റ് അബ്സ്ട്രാക്ഷൻ.
- `rich`: നോട്ട്ബുക്ക് ഔട്ട്പുട്ട് കൂടുതൽ വ്യക്തമായി പ്രിന്റ് ചെയ്യാൻ.

പുനരാരംഭിക്കുന്നത് സുരക്ഷിതമാണ് (ഐഡംപൊട്ടന്റ്). നിങ്ങളുടെ പരിസ്ഥിതിയിൽ ഇവ ഇതിനകം ഉണ്ടെങ്കിൽ ഒഴിവാക്കുക.


In [1]:
# Install required libraries (idempotent)
%pip install -q foundry-local-sdk openai rich


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Explanation: Core Imports
നോട്ട്ബുക്കിൽ മുഴുവൻ ഉപയോഗിക്കുന്ന മോഡ്യൂളുകൾ കൊണ്ടുവരുന്നു:
- ലോക്കൽ മോഡൽ റൺടൈമുമായി ഇന്റർഫേസ് ചെയ്യാൻ `FoundryLocalManager`.
- പരിചിതമായ ചാറ്റ് പൂർത്തീകരണ API സർഫേസ് വീണ്ടും ഉപയോഗിക്കാൻ `OpenAI` ക്ലയന്റ്.
- സ്റ്റൈലുള്ള ഔട്ട്പുട്ടിനായി `rich.print`.

ഇവിടെ യാതൊരു നെറ്റ്‌വർക്ക് കോൾസും നടക്കാറില്ല—ഇത് വെറും നെയിംസ്പേസ് തയ്യാറാക്കുകയാണ്.


In [2]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

### Explanation: Manager Initialization & Metadata Patch
തിരഞ്ഞെടുത്ത അലിയാസിനായി `FoundryLocalManager` ആരംഭിക്കുകയും `promptTemplate` `null` ആകാമെന്നുള്ള സേവന പ്രതികരണങ്ങളെ സുഖപ്രദമായി കൈകാര്യം ചെയ്യാൻ ഒരു പ്രതിരോധ മങ്കി-പാച്ച് പ്രയോഗിക്കുകയും ചെയ്യുന്നു.

പ്രധാന ഫലങ്ങൾ:
- സേവന നിലയും എന്റ്പോയിന്റും സ്ഥിരീകരിക്കുന്നു.
- കാഷ് ചെയ്ത മോഡലുകൾ പട്ടികപ്പെടുത്തുന്നു (ലോകൽ സ്റ്റോർ പരിശോധിക്കുന്നു).
- അലിയാസിനുള്ള കങ്ക്രീറ്റ് മോഡൽ ഐഡി പരിഹരിക്കുന്നു (പിന്നീട് ചാറ്റ് കോൾസിൽ ഉപയോഗിക്കുന്നു).

റോ സർവീസ് മെറ്റാഡാറ്റയിൽ സാധുതാ പ്രശ്നങ്ങൾ നേരിടുന്നുവെങ്കിൽ, SDK ഫോർക്ക് ചെയ്യാതെ എങ്ങനെ ശുദ്ധീകരിക്കാമെന്ന് ഈ മാതൃക കാണിക്കുന്നു.


In [3]:
# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:50262/v1

Cached models:
[
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-generic-gpu:4',
        version='4',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-generic-gpu/versions/4',
        file_size_mb=3809,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-generic-gpu:3',
        version='3',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-generic-gpu/versions/3',
        file_size_mb=700,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-generic-gpu:1',
        version='1',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-generic-gpu/versions/1',
        file_size_mb=2211,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    )
]

Using model id: Phi-4-mini-instruct-generic-gpu:4

### Explanation: അടിസ്ഥാന ചാറ്റ് പൂർത്തീകരണം
ലോകൽ Foundry എന്റ്പോയിന്റിനെ സൂചിപ്പിക്കുന്ന `OpenAI`-സമാനമായ ക്ലയന്റ് സൃഷ്ടിച്ച് ഒറ്റത്തവണ സ്റ്റ്രീമിംഗ് ഇല്ലാത്ത ചാറ്റ് പൂർത്തീകരണം നടത്തുന്നു. ഇവിടെ ശ്രദ്ധിക്കുക:
- മോഡൽ പിശക് കൂടാതെ പ്രതികരിക്കുന്നുണ്ടെന്ന് ഉറപ്പാക്കുക.
- ലാറ്റൻസി / ഔട്ട്പുട്ട് ഫോർമാറ്റ് സാധുവാണെന്ന് പരിശോധിക്കുക.
- വിഭവങ്ങൾ സംരക്ഷിക്കാൻ `max_tokens` മിതമായിരിക്കണം.

ഇത് പരാജയപ്പെടുകയാണെങ്കിൽ, Foundry Local സേവനം പ്രവർത്തിക്കുന്നുണ്ടോ എന്ന് വീണ്ടും പരിശോധിക്കുക, കൂടാതെ ആലിയാസ് ശരിയായി പരിഹരിക്കുന്നുണ്ടോ എന്ന് ഉറപ്പാക്കുക.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':prompt}],
    max_tokens=120,
    temperature=0.5
)
print(resp.choices[0].message.content)

Local inference for privacy refers to the practice of performing data analysis on a local device without sending 
sensitive information to a central server. Two benefits of this approach are:


1. **Enhanced Privacy**: Local inference keeps personal data on the user's device, reducing the risk of data 
breaches and unauthorized access. Since the data is not transmitted over the network, it is less susceptible to 
interception by malicious actors.


2. **Data Sovereignty**: Users retain control over their data, as it does not leave their device. This means that 
individuals or organizations can comply with local data protection regulations, such as the General

### വിശദീകരണം: സ്റ്റ്രീമിംഗ് ചാറ്റ് പൂർത്തീകരണം
കുറഞ്ഞ അനുഭവ വൈകിയതും ഇന്ററാക്ടീവ് UX-ഉം മെച്ചപ്പെടുത്താൻ ടോക്കൺ സ്റ്റ്രീമിംഗ് കാണിക്കുന്നു. ലൂപ്പ് വരുമ്പോൾ ഇൻക്രീമെന്റൽ ഡെൽറ്റകൾ പ്രിന്റ് ചെയ്യുന്നു:
- പ്രാരംഭ ഭാഗിക ഔട്ട്പുട്ട് പ്രധാനപ്പെട്ട ചാറ്റ് UI-കൾക്ക് ഉപകാരപ്രദം.
- ടോക്കൺ ത്രൂപുട്ടും പൂർണ്ണ പൂർത്തീകരണ വൈകിയും തമ്മിലുള്ള അളവ് ചെയ്യാൻ സഹായിക്കുന്നു.

നിങ്ങൾക്ക് ഈ മാതൃക ടോക്കണുകൾ സമാഹരിക്കാൻ, പ്രോഗ്രസ് വിഡ്ജറ്റ് അപ്ഡേറ്റ് ചെയ്യാൻ, അല്ലെങ്കിൽ മിഡ്-ജനറേഷനിൽ ഉപേക്ഷിക്കാൻ ഉപയോഗിക്കാം.


In [5]:
# Streaming example
stream = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}],
    stream=True,
    max_tokens=60,
    temperature=0.4
)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

latency

and

bandwidth

usage

.

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാപത്രം**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, യന്ത്രം ചെയ്ത വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖ അധികാരപരമായ ഉറവിടമായി കണക്കാക്കണം. നിർണായക വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
